In [62]:
import pandas as pd
import json
import ast
from tqdm import tqdm


In [14]:
puid_df = pd.read_csv('../data/p9445_2024.csv')
puid_df['keys'] = puid_df['keys'].apply(ast.literal_eval)
puid_df['values'] = puid_df['values'].apply(ast.literal_eval)
puid_df['meta'] = puid_df['meta'].apply(ast.literal_eval)
meta = list(map(json.loads, puid_df['meta'][0]))
titles = list(map(
    lambda x: f'{"Прямое" if  x["direction"] == 1 else "Обратное"}, полоса {x["lane"]}', meta))
keys = puid_df['keys'][0][0]
general_key_indices = [keys.index('volume'), keys.index(
        'speed'), keys.index('occupancy')]
puid_df = puid_df.set_index('ts')

In [83]:
general_key_indices

[0, 8, 9]

In [80]:
len(keys)

11

In [87]:
puid_df.index = pd.to_datetime(puid_df.index)

In [130]:
def get_processor(source_index):
    def process_keys(row):
        if (len(row['meta']) != len(meta)):
            print(row)
            return pd.Series(index=map(lambda i: keys[i], general_key_indices))
        # if (len(values) != len(meta)):
        #     print(values)
        # if (values[source_index] != len(keys)):
        #     print(values)
        return pd.Series(map(lambda i: pd.to_numeric(row['values'][source_index][i], errors='coerce'), general_key_indices), index=map(lambda i: keys[i], general_key_indices))

    return process_keys

sources_df = []

for (source, i) in tqdm(zip(meta, range(len(meta)))):
    params_df = puid_df.apply(get_processor(i), axis=1)
    sources_df.append(params_df.set_index(puid_df.index))

0it [00:00, ?it/s]

0it [33:44:31, ?it/s]


KeyboardInterrupt: 

In [46]:
list(range(len(meta)))

[0, 1, 2, 3]

count    62749.000000
mean         4.134424
std          2.027946
min          1.000000
25%          4.000000
50%          4.000000
75%          4.000000
max        128.000000
Name: values, dtype: float64

In [89]:
pd.Timestamp('2024-07-19 11:11:01')

Timestamp('2024-07-19 11:11:01')

In [128]:
broken_row = puid_df[pd.to_datetime('2024-07-19 11:05:00'):][:1]

In [129]:
broken_row['meta'].item()

['{"direction":1,"lane":1}',
 '{"direction":1,"lane":2}',
 '{"direction":0,"lane":1}',
 '{"direction":0,"lane":2}']

In [88]:
i = 0
source_index = 1

get_processor(1)(puid_df[pd.Timestamp('2024-07-19 11:11:01')])

KeyError: Timestamp('2024-07-19 11:11:01')

In [61]:
for i in tqdm.tqdm(list(range(len(meta)))):
    print(get_processor(i)(puid_df.iloc[7]))

100%|██████████| 4/4 [00:00<00:00, 322.41it/s]

volume       35.000000
speed        83.546875
occupancy     3.691406
dtype: float64
volume        0.000000
speed        29.660156
occupancy     0.000000
dtype: float64
volume       44.000000
speed        71.980470
occupancy     5.257812
dtype: float64
volume       20.00000
speed        92.63672
occupancy     1.93750
dtype: float64


In [28]:
sources_df[0]

,volume,speed,occupancy
ts,,,
2024-05-10 05:20:00,30.0,78.53125,3.847656
2024-05-10 05:25:00,30.0,80.55469,3.105469
2024-05-10 05:30:00,44.0,78.80859,4.628906
2024-05-10 05:35:00,40.0,75.41797,4.609375
2024-05-10 05:40:00,36.0,86.62891,4.082031
...,...,...,...
2024-11-19 08:00:00,34.0,74.71484,4.644531
2024-11-19 08:05:00,34.0,80.55469,4.707031
2024-11-19 08:10:00,35.0,77.02344,3.992188
